In [1]:
import tensorflow as tf

x = tf.Variable(3, name = "X")
y = tf.Variable(4, name = "Y")
f = x*x*y +y +2

In [4]:
with tf.Session() as sess:
    sess.run(x.initializer)
    sess.run(y.initializer)
    result = f.eval()

In [5]:
print(result)

42


In [6]:
init = tf.global_variables_initializer()

with tf.Session() as sess:
    init.run()
    result = f.eval()

In [7]:
x1 = tf.Variable(1)
x1.graph is tf.get_default_graph()

True

In [10]:
graph = tf.Graph()

with graph.as_default():
    x2 = tf.Variable(2)
    
x2.graph is graph

True

In [11]:
x2.graph is tf.get_default_graph()

False

In [12]:
w = tf.constant(3)
x = w+2
y = x+5
z = x*3

# 실행 시 w,x 값을 두번 평가 실시 -> 효율적이지 않음
with tf.Session() as sess:
    print(y.eval())
    print(z.eval())

10
15


In [14]:
# 실행 시 w,x 값을 한번만 평가 -> 위의 코드 보단 효율적
with tf.Session() as sess:
    y_val, z_val = sess.run([y,z])
    print(y_val)
    print(z_val)

10
15


In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing

housing = fetch_california_housing()
m, n = housing.data.shape
housing_data_plus_bias = np.c_[np.ones((m,1)), housing.data]

x = tf.constant(housing_data_plus_bias, dtype = tf.float32, name = "X")
y = tf.constant(housing.target.reshape(-1, 1), dtype = tf.float32, name = "Y")
xT = tf.transpose(x)
theta = tf.matmul(tf.matmul(tf.matrix_inverse(tf.matmul(xT, x)), xT), y)

with tf.Session() as sess:
    theta_value = theta.eval()
    print(x)
    print(y)
    print(m,n)

Tensor("X:0", shape=(20640, 9), dtype=float32)
Tensor("Y:0", shape=(20640, 1), dtype=float32)
20640 8


In [2]:
print(theta_value)

[[-3.7185181e+01]
 [ 4.3633747e-01]
 [ 9.3952334e-03]
 [-1.0711310e-01]
 [ 6.4479220e-01]
 [-4.0338000e-06]
 [-3.7813708e-03]
 [-4.2348403e-01]
 [-4.3721911e-01]]


In [1]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# 경사 하강법 구현
n_epochs = 1000
learning_rate = 0.01

# 정규화 했다는 가정

x = tf.constant(scaled_housing_data_plus_bias, dtype = tf.float32, name = "X") 
y = tf.constant(housing.target.reshape(-1,1), dtype = tf.float32, name = "Y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(x, theta, name = "predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    
    for epoch in range(n_epochs):
        if epoch % 100 == 0:
            print("Epoch", epoch, "MSE = ", mse.eval())
        sess.run(training_op)
        
    best_theta = theta.eval()



Epoch 0 MSE =  11.724651
Epoch 100 MSE =  0.7063486
Epoch 200 MSE =  0.5849035
Epoch 300 MSE =  0.57113737
Epoch 400 MSE =  0.5621224
Epoch 500 MSE =  0.5550013
Epoch 600 MSE =  0.54929835
Epoch 700 MSE =  0.54471046
Epoch 800 MSE =  0.54100543
Epoch 900 MSE =  0.53800315


In [2]:
print(best_theta)

[[ 2.0685525 ]
 [ 0.92537576]
 [ 0.14905752]
 [-0.4233394 ]
 [ 0.42603254]
 [ 0.00527846]
 [-0.04393153]
 [-0.5890409 ]
 [-0.5697233 ]]


In [3]:
# 미니 배치 경사 하강법을 구현하기 위한 placeholder 함수 예제
A = tf.placeholder(tf.float32, shape = (None, 3))
B = A + 5

with tf.Session() as sess:
    B_val_1 = B.eval(feed_dict = {A: [[1,2,3]]})
    B_val_2 = B.eval(feed_dict = {A: [[4,5,6], [7,8,9]]})
    
print(B_val_1)
print(B_val_2)

[[6. 7. 8.]]
[[ 9. 10. 11.]
 [12. 13. 14.]]


In [5]:
import tensorflow as tf
import numpy as np
from sklearn.datasets import fetch_california_housing
from sklearn.preprocessing import StandardScaler

housing = fetch_california_housing()
m, n = housing.data.shape

scaler = StandardScaler()
scaled_housing_data = scaler.fit_transform(housing.data)
scaled_housing_data_plus_bias = np.c_[np.ones((m, 1)), scaled_housing_data]

# 경사 하강법 구현
n_epochs = 1000
learning_rate = 0.01

# 정규화 했다는 가정

x = tf.placeholder(tf.float32, shape=(None, n + 1), name="X")
y = tf.placeholder(tf.float32, shape=(None, 1), name="Y")
theta = tf.Variable(tf.random_uniform([n+1, 1], -1.0, 1.0), name = "theta")
y_pred = tf.matmul(x, theta, name = "predictions")

error = y_pred - y
mse = tf.reduce_mean(tf.square(error), name = "mse")
optimizer = tf.train.GradientDescentOptimizer(learning_rate = learning_rate)
training_op = optimizer.minimize(mse)

init = tf.global_variables_initializer()

batch_size = 100
n_batches = int(np.ceil(m / batch_size))

def fetch_batch(epoch, batch_index, batch_size):
    np.random.seed(epoch * n_batches + batch_index)  # not shown in the book
    indices = np.random.randint(m, size=batch_size)  # not shown
    X_batch = scaled_housing_data_plus_bias[indices] # not shown
    y_batch = housing.target.reshape(-1, 1)[indices] # not shown
    return X_batch, y_batch

with tf.Session() as sess:
    sess.run(init)

    for epoch in range(n_epochs):
        for batch_index in range(n_batches):
            X_batch, y_batch = fetch_batch(epoch, batch_index, batch_size)
            sess.run(training_op, feed_dict={x: X_batch, y: y_batch})

    best_theta = theta.eval()


In [6]:
print(best_theta)

[[ 2.0714476 ]
 [ 0.8462012 ]
 [ 0.11558535]
 [-0.26835832]
 [ 0.32982782]
 [ 0.00608358]
 [ 0.07052915]
 [-0.87988573]
 [-0.8634251 ]]
